In [1]:
import numpy as np
import plotly.express as px
import plotly.io as pio

from src.turbofan import Turbofan
from utils.configs import (
    config_ex22,
    config_ex23,
    config_teste1,
    config_teste2,
    config_turbofan,
)

pio.templates.default = "plotly"

### Validação 1: Exemplo 2.2

In [2]:
# turbofan = Turbofan(config_ex22)
# turbofan.set_air_flow(533)
# turbofan.print_outputs()

### Teste 1 de PRP-38

In [3]:
# turbofan = Turbofan(config_teste1)
# turbofan.set_air_flow(533)
# turbofan.print_outputs()

### TG: CFM56-7B27

In [7]:
turbofan = Turbofan(config_turbofan)
turbofan.set_air_flow(100)
rated_thrust = 121.4  # kN
fuel_flow = 1.293  # kg/s
optimization_status = turbofan.calibrate_turbofan(rated_thrust, fuel_flow)
# turbofan.plot_calibration_result(fuel_flow / rated_thrust, np.arange(1200,2000, 0.1))
optimization_status

Salvando ponto de projeto...


{'success': True,
 'stage_sequence': ['tsfc_then_thrust'],
 'message_t04': 'Solution found.',
 'message_mdot': 'Solution found.',
 'optimal_t04': 1556.359,
 'optimal_mass_flow_rate': 401.035,
 'final_thrust_kN': 121.4,
 'final_tsfc': 0.01065}

In [15]:
turbofan.update_environment(percentage_of_rated_thrust=7/100)
turbofan.print_outputs()

--- Resultados da Simulação do Motor ---

[ Estações do Motor ]
Estação         | Temp. Total (K)      | Pressão Total (kPa) 
-------------------------------------------------------------
2 (Inlet)       | 288.150              | 101.330             
8 (Fan)         | 289.733              | 102.919             
3 (Compressor)  | 523.014              | 597.716             
4 (Câmara)      | 914.251              | 597.716             
5 (Turbina Comp.) | 680.970              | 141.462             
6 (Turbina Fan) | 671.470              | 132.054             

[ Velocidades de Saída ]
Velocidade de Voo (u_0)         : 0.000 m/s
Velocidade Bocal Quente (u_core): 311.516 m/s
Velocidade Bocal Frio (u_fan)   : 50.415 m/s

[ Performance Geral ]
Razão Combustível/Ar (f)        : 0.01015
Empuxo Específico               : 0.67468 kN.s/kg
Consumo Específico (TSFC)       : 0.01504 kg/(kN.s)
N2                                 : 0.612
N1                                 : 0.462
Empuxo Total            

--- Resultados da Simulação do Motor ---

[ Estações do Motor ]
Estação         | Temp. Total (K)      | Pressão Total (kPa) 
-------------------------------------------------------------
2 (Inlet)       | 288.150              | 101.330             
8 (Fan)         | 289.733              | 102.919             
3 (Compressor)  | 523.014              | 597.716             
4 (Câmara)      | 914.251              | 597.716             
5 (Turbina Comp.) | 680.970              | 141.462             
6 (Turbina Fan) | 671.470              | 132.054             

[ Velocidades de Saída ]
Velocidade de Voo (u_0)         : 0.000 m/s
Velocidade Bocal Quente (u_core): 311.516 m/s
Velocidade Bocal Frio (u_fan)   : 50.415 m/s

[ Performance Geral ]
Razão Combustível/Ar (f)        : 0.01015
Empuxo Específico               : 0.67468 kN.s/kg
Consumo Específico (TSFC)       : 0.01504 kg/(kN.s)
N2                                 : 0.612
N1                                 : 0.462
Empuxo Total            

In [52]:
turbofan.N2_ratio

0.9922637694848851

### Turbofan fora do ponto de projeto

Agora vamos validar como o meta-modelo atua fora do ponto de projeto, confrontando os dados da simulação com os dados reais do motor, obtidos via ICAO.

In [9]:
turbofan = Turbofan(config_ex23)
turbofan.set_sea_level_air_flow(756)
# turbofan.print_outputs()

In [10]:
turbofan.save_design_point()
turbofan.update_from_N2(0.8)
# turbofan.print_config()
turbofan.print_outputs()

Salvando ponto de projeto...
--- Resultados da Simulação do Motor ---

[ Estações do Motor ]
Estação         | Temp. Total (K)      | Pressão Total (kPa) 
-------------------------------------------------------------
2 (Inlet)       | 290.000              | 101.630             
8 (Fan)         | 320.210              | 140.796             
3 (Compressor)  | 646.831              | 1535.262            
4 (Câmara)      | 1166.823             | 1458.499            
5 (Turbina Comp.) | 840.202              | 341.086             
6 (Turbina Fan) | 666.524              | 118.687             

[ Velocidades de Saída ]
Velocidade de Voo (u_0)         : 0.000 m/s
Velocidade Bocal Quente (u_core): 239.269 m/s
Velocidade Bocal Frio (u_fan)   : 237.314 m/s

[ Performance Geral ]
Razão Combustível/Ar (f)        : 0.01325
Empuxo Específico               : 1.67076 kN.s/kg
Consumo Específico (TSFC)       : 0.00793 kg/(kN.s)
N2                                 : 0.800
N1                                 : 

In [11]:
var_and_unit = {
    "N1": "",
    "thrust": "kN",
    "tsfc": "kg/(kN.s)",
    "prf": "",
    "prc": "",
    "pr": "",
    "hot_air_flow": "kg/s",
    "u_core": "m/s",
    "t04": "K",
    "fuel_consumption": "kg/s",
    "bpr": "",
}
for variable, unit in var_and_unit.items():
    if variable in ["prf", "bpr"]:
        param = "N1"
        range_x = [0.2, 1.0]
    else:
        param = "N2"
        range_x = [0.4, 1.0]
    df = turbofan.get_values_by_changing_param(variable, param)
    px.line(
        df,
        x=param,
        y=variable,
        title=f"{variable} vs {param}",
        labels={param: f"{param} (%)", variable: unit},
        range_x=range_x,
        width=800,
        height=400,
    ).show()
    print(round(df.loc[5, variable] / df.loc[0, variable],2))

3.29


21.08


0.76


1.61


4.62


...

inf


C:\Users\andre\AppData\Local\Temp\ipykernel_1168\3802718258.py:32: RuntimeWarning:

divide by zero encountered in scalar divide



1.96


15.96


0.68


In [6]:
from utils.aux_tools import atmosphere
import numpy as np

In [7]:
T,p,rho,mu = atmosphere(35e3*0.3048) # 35k feet

In [20]:
a = np.sqrt(1.4*287*T)
V = a * 0.789
Cd = 315e-4
A = 125

In [22]:
thrust = 1/2*rho*V**2*Cd*A

In [24]:
25/120

0.20833333333333334